In [1]:
from scipy.special import expit
import os
import gzip
import cPickle
import wget
import random
import numpy as np

In [2]:
class NN(object):
    
    def __init__(self, layers):
        self.num_layers = len(layers)
        self.layers = layers
        self.initialize_weights()
    
    def initialize_weights(self):
        self.biases = [np.zeros((y, 1)) for y in self.layers[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(self.layers[:-1], self.layers[1:])]
    
    def forward(self,x):
        activation = x
        activations = [x]
        zs = []    
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softmax(z)
            activations.append(activation)
        return activations, zs
        
        
    def update(self,mini_batch, eta, n ):
        nabla_b = [np.zeroes(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            activations, zs = self.forward(x)
            delta_nabla_b, delta_nabla_w = self.backward(x, y, activations, zs)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
    
    def backward(self, x, y, zs, activations):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        difference = activations[-1] - y
        nabla_b[-1] = difference
        nabla_w[-1] = np.dot(difference, activations[-2].transpose())
        for l in xrange(2, self.num_layers -2, 0, -1):
            dell = np.dot(self.weights[l+1].transpose(), difference) * ( softmax(zs[1])*(1-softmax(zs[1])) ) 
            nabla_b[-l] = difference
            nabla_w[-l] = np.dot(difference, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
        
    def softmax(z):
        e = np.exp(float(z))
        return (e/np.sum(e))

    def loss(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    def train(self,training_data, mini_batch_size, epochs, eta):
        n = len(training_data)
        for epoch in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, minibatches)]
            for mini_batch in mini_batches:
                self.update( mini_batch, eta, len(training_data))

        
        
    #def test(self):
        

In [3]:
def vectorized_result(y):
    e = np.zeros((10, 1))
    e[y] = 1.0
    return e

def load_mnist():
    if not os.path.exists(os.path.join(os.curdir, 'data')):
        os.mkdir(os.path.join(os.curdir, 'data'))
        wget.download('http://deeplearning.net/data/mnist/mnist.pkl.gz', out='data')

    data_file = gzip.open(os.path.join(os.curdir, 'data', 'mnist.pkl.gz'), 'rb')
    training_data, validation_data, test_data = cPickle.load(data_file)
    data_file.close()

    training_inputs = [np.reshape(x, (784, 1)) for x in training_data[0]]
    training_results = [vectorized_result(y) for y in training_data[1]]
    training_data = zip(training_inputs, training_results)

    validation_inputs = [np.reshape(x, (784, 1)) for x in validation_data[0]]
    validation_results = validation_data[1]
    validation_data = zip(validation_inputs, validation_results)

    test_inputs = [np.reshape(x, (784, 1)) for x in test_data[0]]
    test_data = zip(test_inputs, test_data[1])
    return training_data, validation_data, test_data
train_data, valid_data, test_data = load_mnist()

In [4]:
mymodel = NN([784,40,40,10])

In [ ]:
mymodel.train(train_data, 32, 10, 0.01)